In [1]:
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df = pd.read_csv('austin_weather.csv')
# df.shape

In [3]:
#preprocessing on the dataset

df['Date'] = pd.to_datetime(df['Date'])
df['Day'] = pd.DatetimeIndex(df['Date']).day
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['Month'] = pd.DatetimeIndex(df['Date']).month
df.drop(columns='Date',inplace=True)

columns_with_symbol = []

for col in df.columns:
    if df[col].apply(str).str.contains('-', na=False).any():
        columns_with_symbol.append(col)

print(columns_with_symbol)
for col in columns_with_symbol:
    df[col] = df[col].replace('-','')
    df[col] = pd.to_numeric(df[col])

df['PrecipitationSumInches'] = df['PrecipitationSumInches'].replace('T', '')
df['PrecipitationSumInches'] = pd.to_numeric(df['PrecipitationSumInches'])
encoder = LabelEncoder()

print(df)
df['Events'] = encoder.fit_transform(df['Events'])

df.dropna(inplace = True)
# df.dtypes

['DewPointHighF', 'DewPointAvgF', 'DewPointLowF', 'HumidityHighPercent', 'HumidityAvgPercent', 'HumidityLowPercent', 'SeaLevelPressureHighInches', 'SeaLevelPressureAvgInches', 'SeaLevelPressureLowInches', 'VisibilityHighMiles', 'VisibilityAvgMiles', 'VisibilityLowMiles', 'WindHighMPH', 'WindAvgMPH', 'WindGustMPH']
      TempHighF  TempAvgF  TempLowF  DewPointHighF  DewPointAvgF  \
0            74        60        45           67.0          49.0   
1            56        48        39           43.0          36.0   
2            58        45        32           31.0          27.0   
3            61        46        31           36.0          28.0   
4            58        50        41           44.0          40.0   
...         ...       ...       ...            ...           ...   
1314        103        89        75           71.0          67.0   
1315        105        91        76           71.0          64.0   
1316        107        92        77           72.0          64.0   
1317

In [4]:

X = df.drop(['PrecipitationSumInches'], axis=1)
y = df['PrecipitationSumInches']

print(X)

sc = StandardScaler()
x_std = sc.fit_transform(X)
x_std = pd.DataFrame(x_std, columns=X.columns)

# x_std.describe()

      TempHighF  TempAvgF  TempLowF  DewPointHighF  DewPointAvgF  \
0            74        60        45           67.0          49.0   
1            56        48        39           43.0          36.0   
2            58        45        32           31.0          27.0   
3            61        46        31           36.0          28.0   
5            57        48        39           39.0          36.0   
...         ...       ...       ...            ...           ...   
1314        103        89        75           71.0          67.0   
1315        105        91        76           71.0          64.0   
1316        107        92        77           72.0          64.0   
1317        106        93        79           70.0          68.0   
1318         99        88        77           66.0          61.0   

      DewPointLowF  HumidityHighPercent  HumidityAvgPercent  \
0             43.0                 93.0                75.0   
1             28.0                 93.0                68

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_std, y)

print(x_train.columns)

reg = linear_model.LinearRegression()
reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

print('Mean absolute error = ' + str(mean_absolute_error(y_test, y_pred)))
print('Mean squared error = ' + str(mean_squared_error(y_test, y_pred)))
print('R2 score = ' + str(r2_score(y_test, y_pred)))

Index(['TempHighF', 'TempAvgF', 'TempLowF', 'DewPointHighF', 'DewPointAvgF',
       'DewPointLowF', 'HumidityHighPercent', 'HumidityAvgPercent',
       'HumidityLowPercent', 'SeaLevelPressureHighInches',
       'SeaLevelPressureAvgInches', 'SeaLevelPressureLowInches',
       'VisibilityHighMiles', 'VisibilityAvgMiles', 'VisibilityLowMiles',
       'WindHighMPH', 'WindAvgMPH', 'WindGustMPH', 'Events', 'Day', 'Year',
       'Month'],
      dtype='object')
Mean absolute error = 0.18939763672312884
Mean squared error = 0.13213965939564384
R2 score = 0.33528030286490906


In [6]:
# joblib.dump(reg, './model_new.pkl')